trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae

https://translate.yandex.net/api/v1.5/tr/translate
 ? key=<API key>
 & text=<text to translate>
 & lang=<translation direction>
 & [format=<text format>]
 & [options=<translation options>]

In [1]:
import requests
from xml.etree import ElementTree

r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                  data={'key': 'trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae',\
                        'text': 'he is a human',\
                        'lang': 'en-pt',\
                        'options': '4'})

temp = ElementTree.fromstring(r.text)
print(ElementTree.fromstring(r.text), r.text)

<Element 'Translation' at 0x7f55d584b098> <?xml version="1.0" encoding="utf-8"?>
<Translation code="200" lang="en-pt"><text>ele é um ser humano</text><align>0:13-0:19,0:2-0:3,3:2-4:1,6:7-6:13</align></Translation>


In [2]:
temp[0].text

'ele é um ser humano'

### Translating

In [3]:
from pathlib import Path
import os
import re

In [4]:
os.chdir('../')

In [5]:
BASE_PATH = Path.cwd()
TRAIN_PATH = BASE_PATH / 'data/WSD_Evaluation_Framework/Training_Corpora/SemCor+OMSTI/semcor+omsti.data.xml'
REMOVED_WORDS = ['&', '\n']


In [6]:
def clean_word(word):
    for special_word in REMOVED_WORDS:
        if special_word in word:
            return False
    return True

In [7]:
def load_senses(path):
    map_wordnet = {}
    t = []
    with open(path) as file:
        for line in file.read().split('\n'):
            if line == '':
                continue
            terms = line.split()
            map_wordnet[terms[0]] = terms[1:]
            
    return map_wordnet

In [8]:
def convert_to_sentence_id(sentence):
    tokens = []
    token_id = 0
    for temp in sentence:
        if clean_word(temp):
            for word in temp.split():
                tokens.append(token_id)
            token_id += 1
            
    return tokens

In [9]:
def get_tagged_sentence(sentence, tags_sense, map_wordnet):
    final_sentence = []
    for ix, tag in enumerate(tags_sense):
        temp = []
        #print(sentence[ix])
        if clean_word(sentence[ix]):
            if tag != 'no_instance':
                senses = map_wordnet[tag]
                if len(senses) == 1:
                    temp = senses[0]
                else:
                    # Replace this code
                    temp = senses[0]
            else:
                temp = sentence[ix]
            final_sentence.append(temp)
        
    return ' '.join(final_sentence)

In [ ]:
with open(TRAIN_PATH) as f:
    xml = f.read()
    map_wordnet = load_senses(BASE_PATH / 'data/WSD_Evaluation_Framework/Training_Corpora/SemCor+OMSTI/semcor+omsti.gold.key.txt')
    
    context = re.findall(r'<sentence(.*?)</sentence>', xml, re.DOTALL)
    word_ambiguos = re.findall(r'<head>(.*?)</head>', context[0], re.DOTALL)

    sentences = re.split(r'[\.|:|?|!]', context[0])
    
    list_instance = re.findall(r'<sentence(.*?)</sentence>', xml, re.DOTALL)
    input_sentences = []
    target_sentences = []
    sentences_to_id = []

    for ix, instance in enumerate(list_instance):
        # Constructing train sentences
        #print(ix)
        
        sentence = re.findall(r'>(.*?)<', instance, re.DOTALL)
        input_sentences.append(' '.join([word for word in sentence if clean_word(word)]))     
        sentences_to_id.append(convert_to_sentence_id(sentence))
        
        # Extracting tags wf or instance
        labels = re.findall(r'</(.*?)>', instance, re.DOTALL)

        # Mapping senses ids
        tags_sense = re.findall(r'instance id="(.*?)"', instance, re.DOTALL)
        tags_sense.reverse()
        tags_sense = [tags_sense.pop() if label == 'instance' else 'no_instance' for index, label in enumerate(labels)]

        # Constructing test sentences
        #target_sentences.append(get_tagged_sentence(sentence, tags_sense, map_wordnet))
        target_sentences.append(tags_sense)

In [ ]:
api_requests = []

In [ ]:
from tqdm import tqdm


for sentence in tqdm(input_sentences[85522:]):
    r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                      data={'key': 'trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae',\
                            'text': f'{sentence}',\
                            'lang': 'en-pt',\
                            'options': '4'})
    api_requests.append(r)
    


  0%|          | 5/765452 [00:06<287:45:54,  1.35s/it]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/krivas/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/krivas/anaconda3/lib/python3.6/site-packages/tqdm-4.19.5-py3.6.egg/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/krivas/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 235/765452 [04:45<258:06:18,  1.21s/it]

In [14]:
len(input_sentences)

850974

In [13]:
len(api_requests)

85522

In [12]:
print(8)

8


In [141]:
import random

ix = random.randint(0,9000)
print(ix)
len(input_sentences[ix].split()), len(target_sentences[ix])

7585


(18, 17)

In [147]:
import random

ix = random.randint(0,9000)
print(ix)
len(input_sentences[ix].split()), len(target_sentences[ix])

2690


(14, 13)

In [153]:
import random

ix = random.randint(0,9000)
print(ix)
len(input_sentences[ix].split()), len(target_sentences[ix])

5107


(58, 55)

In [150]:
target_sentences[ix]

['no_instance',
 'd023.s038.t000',
 'no_instance',
 'd023.s038.t001',
 'no_instance',
 'd023.s038.t002',
 'no_instance',
 'd023.s038.t003',
 'no_instance',
 'd023.s038.t004',
 'no_instance',
 'no_instance',
 'no_instance']

In [152]:
input_sentences[ix].split()

['He',
 'opened',
 'the',
 'door',
 'and',
 'went',
 'in',
 ',',
 'pulling',
 'it',
 'shut',
 'behind',
 'him',
 '.']

In [54]:
len(sentence), len(labels)

(27, 27)

In [55]:
tags_sense

[]

In [51]:
labels

['wf',
 'wf',
 'instance',
 'wf',
 'wf',
 'instance',
 'wf',
 'instance',
 'instance',
 'instance',
 'wf',
 'wf',
 'wf',
 'instance',
 'wf',
 'instance',
 'wf',
 'instance',
 'instance',
 'instance',
 'wf',
 'wf',
 'instance',
 'wf',
 'instance',
 'instance',
 'wf']

In [49]:
instance

' id="d000.s002">\n<wf lemma="what" pos="DET">What</wf>\n<instance id="d000.s002.t000" lemma="effort" pos="NOUN">effort</instance>\n<wf lemma="do" pos="VERB">do</wf>\n<wf lemma="you" pos="PRON">you</wf>\n<instance id="d000.s002.t001" lemma="make" pos="VERB">make</instance>\n<wf lemma="to" pos="PRT">to</wf>\n<instance id="d000.s002.t002" lemma="assess" pos="VERB">assess</instance>\n<instance id="d000.s002.t003" lemma="result" pos="NOUN">results</instance>\n<wf lemma="of" pos="ADP">of</wf>\n<wf lemma="you" pos="PRON">your</wf>\n<instance id="d000.s002.t004" lemma="program" pos="NOUN">program</instance>\n<wf lemma="?" pos=".">?</wf>\n'

In [45]:
tags_sense

[None,
 'd000.s000.t000',
 None,
 None,
 'd000.s000.t001',
 None,
 None,
 'd000.s000.t002',
 None,
 'd000.s000.t003',
 None,
 None,
 'd000.s000.t004',
 None,
 'd000.s000.t005',
 'd000.s000.t006',
 None]

In [46]:
labels

['wf',
 'instance',
 'wf',
 'wf',
 'instance',
 'wf',
 'wf',
 'instance',
 'wf',
 'instance',
 'wf',
 'wf',
 'instance',
 'wf',
 'instance',
 'instance',
 'wf']

In [36]:
tags_sense.pop()

'd000.s000.t006'

In [38]:
tags_sense.reverse()

In [39]:
tags_sense

['d000.s000.t005',
 'd000.s000.t004',
 'd000.s000.t003',
 'd000.s000.t002',
 'd000.s000.t001',
 'd000.s000.t000']

In [42]:
sentence

['What',
 'effort',
 'do',
 'you',
 'make',
 'to',
 'assess',
 'results',
 'of',
 'your',
 'program',
 '?']

In [43]:
len(sentence), len(tags_sense)

(12, 17)